# **DockerFile Creation**

| | |
|-|-|
| Author(s) | [Keeyana Jones](https://github.com/keeyanajones/) |

## **Overview**

A Dockerfile is a plain text file that contains a series of instructions that Docker uses to automatically build a Docker image.  Its essentially a blueprint or a recipe for creating a reproducible and portable environment for your application.  Each instruction in a Dockerfile creates a layer in the Docker image, making builds efficient and allowing for caching. 

### 

- **Reproducibility**  Ensures that anyone building the image from the Dockerfile will get the exact same environment, regardless of their local machine setup.
- **Portability:** The resulting Docker image can be run consistently on any Docker compatible environment (local machine, cloud, server).
- **Isolation:** your application runs in a self contained environment, separate form the host system and other applications.
- **Version Control:** Dockerfiles are text files, so they can be easily version controlled using systems like git, allowing you to track changes to your build process.
- **Efficiency:** Dockers layered approach and cahing mechanisms make rebuilding images fast when only small changes occur.  

### **Basic Structure and Key Instructions**

A Dockerfile typically starts with a base image and then adds layers of configurations, dependencies, and application code.  Here are the most common instructions: 
1. `FROM`:
   - **Purpose:** Specifies the base image for your build.  This is usually an official image from Docker Hub (e.g., `ubuntu`, `python`, `node`, `nginx`), it must be the first instruction in a Dockerfile.
   - **Example:** `FROM python:3.9-slim-buster` (uses a lightweight Python 3.9 image based on Debian Buster)   

2. `WORKDIR`: 
   - **Purpose:** Sets the working directory fro any `RUN`, `CMD`, `ENTRYPOINT`, `COPY`, or `ADD` instructions that follow it.  If the directory doesn't exist, it will be created.
   - **Example:** `WORKDIR /app` (all subsequent commands will execute relative to `/app` inside the image). 

3. `COPY`:
   - **Purpose:** Copies new files or directories from the host machine (where your building the image) into the Docker images filesystem.  Its generally preferred over for simple copying.
   - **Example:** `COPY . .` (copies everything from the current directory on the host to the `WORKDIR` inside the image)
   - **Example:** `COPY requirements.txt .` (copies only `requirements.txt`).

4. `RUN`:
   - **Purpose:** Executes commands in a new layer on top of the current image and commits the result.  Used for installing software, creating directories, running build scripts, etc.
   - **Example:** `RUN opt-get update && opt-get install -y git` (updates package and installs git)
   - **Example:** `RUN pop install -f requirements.txt` (installs Python dependencies)

5. `EXPOSE`:
   - **purpose:** informs Docker that the container listens on the specified network ports at runtime.  It doesn't actually publish the port; its more of a documentation and networking hint. You need to use the `-p` flag with `docker run` to publish the port.
   - **Example:** `EXPOSE 8000` (indicates the application inside listens on port 8000)

6. `CMD`:
   - **Purpose:** Provides default commands and arguments for an executing container.  There can only be one `CMD` instruction in a Dockerfile. If you specify an executable, it will be run.  If you provide arguments, they will be appended to teh ENTRYPOINT instruction (if one exists).
   - **Example (executable form):** `CMD ["python", "app.py"]` 
   - **Example (shell form):** `CMD python app.py` (less preferred as it runs through a shell, losing some ignal handling)

7. `ENTRYPOINT`:
   - **Purpose:** Configures a container that will run as an executable.  Like `CMD`, there can only be one `ENTRYPOINT`. If both `ENTRYPOINT` and `CMD` are present, `CMD`s value becomes arguments to the `ENTRYPOINT`.  Often used to set up a fixed command with variable arguments.
   - **Example:** `ENTRYPOINT ["gunicorn", "--bind", "0.0.0.0:8000", "my_app:app"]`
   - **Example:** with CMD:

In [ ]:
ENTRYPOINT ["/usr/bin/supervisord", "-n"]
CMD ["myservice"] # passed as argument to supervisord

### **Example Dockerfile (Python Flask Application)**

Lets image you have a simple Flask app:

`app.py`: 

   - **Purpose:** similar to `.gitignore`, this file specifies files and directories that should be excluded whe the Docker client sends the build context to the Docker daemon. This prevents sending unnecessary files, speeding up the build process and reducing image size.
   - **Example:** `* .pyc`, `.venv/`, `.git/`, `__pycache__/`, `node_modules/`

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello():
    return "Hello from Docker!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

`requirement.txt`:

In [ ]:
Flask==2.3.2

Here's the corresponding `Dockerfile`:

In [ ]:
# Use an official Python runtime as a parent image
FROM python:3.9-slim-buster
# Set the working directory in the container
WORKDIR /app
# Copy the requirements file into the container at /app
COPY requirements.txt .
# Install any needed packages specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt
# Copy the rest of the application code into the container at /app 
COPY . .
# Make port 500 available to the world outside this container 
EXPOSE 5000
# Run app.py when the container launches 
CMD ["python","app.py"]

And a `.dockerignore` file:

In [ ]:
__pycache__/
*.pyc
.git/
.venv/

### **Building and Running an image from a Dockerfile**

1. **Save the Dockerfile:** Save the above content in a file named `Dockerfile` (no extension) in the same directory as `app.py` and `requirements.txt`.
2. **Build the image:** Open your terminal in that directory and run:

In [ ]:
docker build -t my-flask-app .

- **`-t my-flask-app`:** Tags the image with the name `my-flask-app`. 
- **`.`:** Specifies the build context (the current directory), meaning Docker will look for the Dockerfile and related files in the current directory.

3. Run the container:

In [ ]:
docker run -p 5000:5000 my-flask-app

- **`-p 5000:5000`:** maps port 5000 on your host machine to port 5000 inside the container (where your Flask app is listening).
- **`my-flask-app`:** The name of the image to run.

Now, if you open your web browser and go to `http://localhost:5000`, you should see "Hello from Docker!".

### **Dockerfile Best Practices**

- **Small Base Images:** Use lean base images (e.g., -slim, alpine) to reduce image size, which speed up builds, pulls, and reduces attack surface.
- **Layer Caching:** order your instructions form least to most frequently changing.  For example, install dependencies (`RUN pip install`) before copying your application code (`COPY . .`), so Docker can cache the dependency installation step if only your code changes.
- **Minimize Layers:** Combine multiple `RUN` commands into a simple `RUN` command using `&&` to reduce the number of layers and image size.  Clean up temporary files in the same `RUN` command.  
- **Use `.dockerignore`:** Exclude unnecessary files from the build context (e.g., `.git`, `node_modules`, `__pycache__`) to speed up builds and reduce image size.
- **Specific Versions:** pin dependencies to specific versions (e.g.,`python:3.9-slim-buster`, `flask==2.3.2`) for reproducibility.
- **Non-Root User:** Run your application as a non root user inside the container for security reasons (`User` instruction).
- **Multi-stage Builds:** For complex applications, use multistage builds to create smaller, more secure production images by separating build time dependencies from runtime dependencies. 
   - **Example:** Build a Go binary in one stage, then copy only the binary into a `scratch` or `alpine` image in teh final stage.
- **CMD vs. ENTRYPOINT:**
Understand their differences and use them appropriately. `ENTRYPOINT` is good for setting up an executable that `CMD` can pass arguments to. 
- **Health Checks:** Consider adding `HEALTHCHECK` instructions for production containers to tell Docker how to test if a container is still working. 

Mastering Dockerfile creation is a fundamental skill for anyone working with containerized applications, enabling efficient, reproducible, and portable deployments. 

----